ETF DASHBOARD V.2.0

# | Technical Stock Screener
- *Inspired from Mark Minervini's Trend Template*
- *http://www.minervini.com/blog/index.php/blog/first_things_first_how_to_chart_stocks_correctly_and_increase_your_chances.*
    

In [28]:
# Author: Ken Lee 2020.01.12
# Our Library
import ETFFundamentalData as finData
import ETFHistoryDownload as hist
import ETFHistoryDownload as hist
import ETFStockAnalytics as analytic
import ETFPerformanceForecast as perf
# Common Library
import sqlalchemy as sql
import hvplot.pandas
import numpy as np
from pandas import ExcelWriter
import pandas as pd
#import datetime
import time
import os
from pathlib import Path
import logging
from dateutil.relativedelta import relativedelta
from datetime import datetime
# DB Connection
eft_data_connection_string = 'sqlite:///./Resources/etf.db'
our_portfolio_export = Path('./Resources/our_portfolio.csv')
etf_data_engine = sql.create_engine(eft_data_connection_string, echo=True)


## STEP1. HISTORICAL DATA RETRIVAL FROM ETF_DB SQLLITE (V1 DATA)

In [29]:
# RETRIEVE OUR CANDIDATE STOCK POOL BASED ON 12 BENCHMARK ETF (V1 IMPLEMENTATION)

#SPECIFIY REVIEW CUTOFF DATE HERE FOR CREATION AND REVAMP UP 
t_date = datetime(2019, 12, 31) 

dates_list_df = hist.get_hist_record_breakdown_by_period(t_date)
historical_px_matrix = hist.get_price_history_by_period(t_date)
historical_px_matrix 
# 1. Analyze all names performance for comparison
x_start_date = dates_list_df.loc[dates_list_df['period'] == 'Y1', 'date'].iloc[0]
x_end_date = dates_list_df.loc[dates_list_df['period'] == 'M6', 'date'].iloc[0]
y_end_date = t_date 
        
daily_xy_horizon_return_matrix = analytic.get_xy_daily_return_matrix(t_date, x_start_date, x_end_date, y_end_date)
daily_price_matrix = analytic.get_price_matrix(daily_xy_horizon_return_matrix, x_start_date, y_end_date)
daily_return_matrix = analytic.get_daily_return_matrix(daily_price_matrix)
cumulative_returns_matrix =  analytic.get_cumulative_return_matrix(daily_price_matrix)
year_trading_days = 252
rolling_days = 30
analysis_matrix = analytic.get_std_matrix(daily_return_matrix, year_trading_days, rolling_days)
performance_summary_matrix = daily_xy_horizon_return_matrix.merge(analysis_matrix, on = 'symbol', how = 'left')
performance_summary_matrix.head()

,D0,D7_W1,M1,M3,M6,Y0_YTD,Y1,Y2,Y3,Start Date,...,X_End Close,X_Return,Y_End Date,Y_End Close,Y_Return,XY_Return,Annualized_std_dev,Average_annual_return,Sharpe_ratio,SPY_30d_roll_beta
symbol,,,,,,,,,,,,,,,,,,,,,
A,145.17,85.3100,80.36,74.80,75.81,65.6000,67.035,67.605,68.36,2018-12-31,...,75.81,0.130902,2019-12-31,145.17,1.165585,0.914919,0.243667,0.293732,1.205463,1.311107
AAON,74.09,49.8000,48.63,45.78,49.71,34.9600,35.020,37.000,34.45,2018-12-31,...,49.71,0.419475,2019-12-31,74.09,1.115648,0.490445,NaN,NaN,NaN,NaN
AAPL,172.17,284.2762,264.09,224.58,201.68,157.5225,156.700,172.260,172.48,2018-12-31,...,201.68,0.287045,2019-12-31,172.17,0.098724,-0.146321,0.262006,0.659566,2.517373,1.429280
AAT,38.32,44.7000,46.76,46.24,46.88,39.2200,40.150,38.160,38.36,2018-12-31,...,46.88,0.167621,2019-12-31,38.32,-0.045579,-0.182594,NaN,NaN,NaN,NaN
ABBV,136.38,89.8400,87.04,73.23,73.44,89.0800,91.410,96.710,92.31,2018-12-31,...,73.44,-0.196587,2019-12-31,136.38,0.491959,0.857026,0.274851,0.032815,0.119390,1.118133


## STEP2. SMA, MULTIPLES (BY PERIOD), RS(RELATIVE STRENGTH) STAT

In [30]:

# 1) Simple Moving Average and 52 Weeks Low and High
perf_screen_df = pd.DataFrame(daily_price_matrix.iloc[-1])
perf_screen_df.columns[0]
perf_screen_df.rename(columns={perf_screen_df.columns[0]:'CUR_PX'}, inplace = True)
perf_screen_df['SMA_50'] = pd.DataFrame(round(daily_price_matrix.rolling(window=50).mean(),2).iloc[-1])
perf_screen_df['SMA_150'] = pd.DataFrame(round(daily_price_matrix.rolling(window=150).mean(),2).iloc[-1])
perf_screen_df['SMA_200'] = pd.DataFrame(round(daily_price_matrix.rolling(window=200).mean(),2).iloc[-1])
perf_screen_df['SMA_200_20'] = pd.DataFrame(round(daily_price_matrix.rolling(window=200).mean(),2).iloc[-20])
perf_screen_df['HIGH_52WK'] = pd.DataFrame(round(daily_price_matrix.rolling(window=52).max(),2).iloc[-1])
perf_screen_df['LOW_52WK'] = pd.DataFrame(round(daily_price_matrix.rolling(window=52).min(),2).iloc[-1])
# 2) Return Multiple
return_multiple = performance_summary_matrix[['X_Return','XY_Return','Y_Return']].copy()
return_multiple.rename(columns={'X_Return':'H1_Return','XY_Return':'H2_Return','Y_Return':'1Y_Return'}, inplace = True)
return_multiple['H1_Multiple'] = round (( return_multiple['H1_Return'] /  return_multiple.at['SPY','H1_Return'] ), 2)
return_multiple['H2_Multiple'] = round (( return_multiple['H1_Return'] /  return_multiple.at['SPY','H1_Return'] ), 2)
return_multiple['1Y_Multiple'] = round (( return_multiple['1Y_Return'] /  return_multiple.at['SPY','1Y_Return'] ), 2)
display(return_multiple.head())

# 3) RS Rating Relavtive Strong Rating 
# SP 500 Benchmark
display(pd.DataFrame(perf_screen_df.loc['SPY']).transpose())
rs_rating_df = return_multiple[['H1_Multiple','H2_Multiple','1Y_Multiple']].copy()
rs_rating_df.rename(columns={'H1_Multiple':'H1_Rank','H2_Multiple':'H2_Rank','1Y_Multiple':'Y1_Rank'}, inplace = True)
rs_rating_df = round((rs_rating_df.rank(pct=True) * 100), 2)
perf_screen_df['RS_RATING'] = rs_rating_df['Y1_Rank'].copy()
display(perf_screen_df.head())

,H1_Return,H2_Return,1Y_Return,H1_Multiple,H2_Multiple,1Y_Multiple
symbol,,,,,,
A,0.130902,0.914919,1.165585,0.69,0.69,1.33
AAON,0.419475,0.490445,1.115648,2.22,2.22,1.28
AAPL,0.287045,-0.146321,0.098724,1.52,1.52,0.11
AAT,0.167621,-0.182594,-0.045579,0.89,0.89,-0.05
ABBV,-0.196587,0.857026,0.491959,-1.04,-1.04,0.56


,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK
SPY,321.89,311.76,299.67,296.45,292.73,322.91,297.98


,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK,RS_RATING
symbol,,,,,,,,
A,85.29,79.88,74.73,75.22,74.80,85.48,74.56,63.16
AAPL,293.37,265.11,227.11,219.18,209.55,293.37,235.29,11.88
ABBV,88.50,85.51,75.85,76.88,76.02,90.23,75.15,31.92
ABCB,42.51,43.45,39.81,38.78,38.43,44.59,41.43,41.43
ABT,86.84,84.56,83.94,82.35,81.53,87.39,80.54,50.76


## STEP3. FILTER STOCKS BASED ON SCREENING CRITERIA 
- #8. RS-Rating must be greater than [high52wk_within%]
- #1. Current Price > 150 SMA and 
- #2. 150 SMA and > 200 SMA
- #3. 200 SMA trending up for at least 1 month
- #4. 50 SMA> 150 SMA
- #5. 50 SMA> 200 SMA
- #6. Current PX is at least [low52wk_above%] above 52 week low
- #7. Current PX is at least [high52wk_within%] above 52 week low

In [31]:
perf_screen_df2 = perf_screen_df.copy()

rs_pct_cutoff = 0.60
low52wk_above = 1.02
high52wk_within = 0.70

#7. RS-Rating must be greater than [high52wk_within%]
perf_screen_df2 = perf_screen_df2[perf_screen_df2.RS_RATING >= perf_screen_df2.RS_RATING.quantile(rs_pct_cutoff)].dropna()
#1. Current Price > 150 SMA and 
perf_screen_df2 = perf_screen_df2[perf_screen_df2.CUR_PX > perf_screen_df2.SMA_150].dropna()
#2. 150 SMA and > 200 SMA
perf_screen_df2 = perf_screen_df2[perf_screen_df2.SMA_150 > perf_screen_df2.SMA_200].dropna()
#3. 200 SMA trending up for at least 1 month
perf_screen_df2 = perf_screen_df2[perf_screen_df2.SMA_200 > perf_screen_df2.SMA_200_20].dropna()
#4. 50 SMA> 150 SMA
perf_screen_df2 = perf_screen_df2[perf_screen_df2.SMA_50 > perf_screen_df2.SMA_150].dropna()
#4. 50 SMA> 200 SMA
perf_screen_df2 = perf_screen_df2[perf_screen_df2.SMA_50 > perf_screen_df2.SMA_200].dropna()
#5. Current PX is at least [low52wk_above%] above 52 week low
perf_screen_df2 = perf_screen_df2[perf_screen_df2.CUR_PX >= (low52wk_above * perf_screen_df2.LOW_52WK)].dropna()
#6. Current PX is at least [high52wk_within%] above 52 week low
perf_screen_df2 = perf_screen_df2[perf_screen_df2.CUR_PX >= (high52wk_within * perf_screen_df2.HIGH_52WK)].dropna()

## STEP4. RESULT - STOCKS SCREENED

In [32]:
# SELECTED PORTFOLIO (Top 5, Bottom 5)
display(perf_screen_df2.head())
display(perf_screen_df2.tail())
# SUMMARY
perf_screen_df2.describe()
# SAVE AND EXPORT THE PORTFOLIO FOR NEXT STEP
if os.path.exists(our_portfolio_export):
    os.remove(our_portfolio_export)
perf_screen_df2.to_csv(our_portfolio_export)

,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK,RS_RATING
symbol,,,,,,,,
ACN,210.50,197.39,192.53,188.44,184.30,212.24,182.74,78.61
AMAT,61.04,57.66,50.71,48.48,46.51,62.37,50.53,94.23
AMD,45.87,38.87,33.51,31.97,30.13,46.64,30.97,97.96
APPS,7.14,7.73,6.48,5.79,5.35,8.87,6.55,99.49
ARWR,63.43,54.49,37.70,33.09,28.60,73.00,35.94,95.25


,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK,RS_RATING
symbol,,,,,,,,
UPS,117.06,119.06,113.90,112.14,111.40,124.30,114.00,63.84
VICR,46.66,40.08,33.88,33.60,32.59,46.66,30.97,84.72
WST,150.25,147.02,138.50,132.61,128.40,151.14,138.75,93.21
XLK,91.69,87.03,81.83,80.32,78.56,91.93,80.66,79.97
YETI,34.76,31.96,30.28,30.21,29.42,35.04,28.73,95.93


Next: ETFv2_FinancialAnalyzer.ipynb